In [ ]:
import os
import cv2
import json
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycocotools.coco import COCO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/v1.0/InfinityAI_InfiniteRep_fly_v1.0.zip -d

Archive:  /content/drive/MyDrive/v1.0/InfinityAI_InfiniteRep_fly_v1.0.zip
replace data/000030_img_labels.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: data/000030_img_labels.zip  
 extracting: data/000025_img_labels.zip  
  inflating: data/000000.json        
  inflating: data/000038.mp4         
  inflating: data/000010.mp4         
  inflating: data/000057.json        
  inflating: data/000004.mp4         
 extracting: data/000054_img_labels.zip  
 extracting: data/000041_img_labels.zip  
  inflating: data/000041.json        
  inflating: data/000016.json        
  inflating: data/000082.json        
 extracting: data/000049_img_labels.zip  
  inflating: data/000005.mp4         
  inflating: data/000011.mp4         
  inflating: data/000039.mp4         
 extracting: data/000038_img_labels.zip  
  inflating: data/000094.json        
 extracting: data/000022_img_labels.zip  
 extracting: data/000037_img_labels.zip  
  inflating: data/000007.mp4         
  inflating: data/00

In [ ]:
coco = COCO("/content/data/000000.json")
cap = cv2.VideoCapture("/content/data/000000.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
imgs = []
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
    imgs.append(img)
imgs = np.array(imgs)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
image_dims = (224,224)

kp_path = os.path.join("/content/", "keypoints.mp4")
out = cv2.VideoWriter(kp_path, fourcc, fps, image_dims)

for img, img_data in zip(imgs, coco.imgs.values()):
    canvas = img.copy()
    img_id = img_data["id"]
    rep_count = img_data["rep_count"]

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
        # skip non-person instances
        if "armature_keypoints" not in ann:
            continue
        keypoints = ann["armature_keypoints"]
        keyp = []
        for keypoint_name, keypoint_info in keypoints.items():
          x, y, z, v = keypoint_info["x"], keypoint_info["y"], keypoint_info["z"], keypoint_info["v"]
          # print(keypoint_name,":","x:",x,"y:", y,"z:", z,"v:", v)
          keyp.append(keypoint_name)


In [ ]:
unique_values = set(keyp)
print(unique_values)

{'pelvis', 'left_shoulder', 'left_pinky3', 'head', 'right_ring1', 'neck', 'jaw', 'right_wrist', 'right_ring3', 'left_index2', 'right_pinky1', 'right_collar', 'left_ring2', 'right_middle2', 'right_middle3', 'left_middle2', 'left_ring1', 'left_ring3', 'right_index3', 'right_pinky3', 'spine1', 'right_knee', 'spine3', 'left_ankle', 'left_middle3', 'left_thumb3', 'left_thumb1', 'left_hip', 'left_index3', 'left_collar', 'left_middle1', 'right_middle1', 'left_wrist', 'left_pinky1', 'right_index1', 'left_index1', 'spine2', 'left_elbow', 'right_shoulder', 'right_eye_smplhf', 'right_thumb2', 'left_pinky2', 'left_thumb2', 'right_elbow', 'right_thumb3', 'right_foot', 'right_pinky2', 'right_thumb1', 'left_knee', 'right_hip', 'right_index2', 'left_foot', 'right_ankle', 'left_eye_smplhf', 'right_ring2'}


In [ ]:
keypoint_list = list(unique_values)

In [ ]:
 column_names = []
 column_names.append("ID")
 column_names.append("rep_count")
 for i in keypoint_list:
  column_names.append(i+"_x")
  column_names.append(i+"_y")
  column_names.append(i+"_z")
  column_names.append(i+"_v")

In [ ]:
len(column_names)

222

In [ ]:
import pandas as pd

df = pd.DataFrame(columns = column_names )


In [ ]:
exercise_folder = '/content/data'
json_paths = sorted(glob.glob(os.path.join(exercise_folder, "*.json")))
video_paths = sorted(glob.glob(os.path.join(exercise_folder, "*.mp4")))
print(json_paths,video_paths)

['/content/data/000000.json', '/content/data/000001.json', '/content/data/000002.json', '/content/data/000003.json', '/content/data/000004.json', '/content/data/000005.json', '/content/data/000006.json', '/content/data/000007.json', '/content/data/000008.json', '/content/data/000009.json', '/content/data/000010.json', '/content/data/000011.json', '/content/data/000012.json', '/content/data/000013.json', '/content/data/000014.json', '/content/data/000015.json', '/content/data/000016.json', '/content/data/000017.json', '/content/data/000018.json', '/content/data/000019.json', '/content/data/000020.json', '/content/data/000021.json', '/content/data/000022.json', '/content/data/000023.json', '/content/data/000024.json', '/content/data/000025.json', '/content/data/000026.json', '/content/data/000027.json', '/content/data/000028.json', '/content/data/000029.json', '/content/data/000030.json', '/content/data/000031.json', '/content/data/000032.json', '/content/data/000033.json', '/content/dat

In [ ]:
for json,vd in zip(json_paths,video_paths):
  coco = COCO(json)
  cap = cv2.VideoCapture(vd)
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  imgs = []
  while cap.isOpened():
      ret, img = cap.read()
      if not ret:
          break
      imgs.append(img)
  imgs = np.array(imgs)

  for img, img_data in zip(imgs, coco.imgs.values()):
    img_id = img_data["id"]
    rep_count = img_data["rep_count"]
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
      # skip non-person instances
      if "armature_keypoints" not in ann:
        continue
      keypoints = ann["armature_keypoints"]
      row_val=[]
      row_val.append(img_id)
      row_val.append(rep_count)
      for i in keypoint_list:
        for keypoint_name, keypoint_info in keypoints.items():
          if i == keypoint_name:
            x, y, z, v = keypoint_info["x"], keypoint_info["y"], keypoint_info["z"], keypoint_info["v"]
            # print(img_id, keypoint_name,":","x:",x,"y:", y,"z:", z,"v:", v)
            row_val.append(x)
            row_val.append(y)
            row_val.append(z)
            row_val.append(v)
          else:
            continue
      df.loc[len(df)] = row_val


loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.11s)
creating index...
index created!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
load

In [ ]:
df.describe()

,ID,rep_count,pelvis_x,pelvis_y,pelvis_z,pelvis_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,...,right_ankle_z,right_ankle_v,left_eye_smplhf_x,left_eye_smplhf_y,left_eye_smplhf_z,left_eye_smplhf_v,right_ring2_x,right_ring2_y,right_ring2_z,right_ring2_v
count,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,...,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000,22100.000000
mean,353.832217,3.779738,111.085928,137.021312,4.179402,1.985837,120.951267,102.538552,4.234447,1.943620,...,4.243596,1.538281,113.826380,86.365747,4.127070,1.988688,93.598733,118.210950,4.027100,1.982760
std,232.500482,2.427939,43.438973,8.040856,1.061620,0.118165,44.398737,6.597798,1.067495,0.230659,...,1.062882,0.783461,43.462535,8.999436,1.069797,0.105758,44.490372,6.812774,1.075390,0.162065
min,0.000000,0.013251,27.000000,120.000000,2.811358,1.000000,42.000000,87.000000,2.818473,1.000000,...,2.672550,0.000000,30.000000,64.000000,2.806996,1.000000,-18.000000,98.000000,2.422515,0.000000
25%,165.000000,1.755837,79.000000,132.000000,3.392512,2.000000,93.000000,97.000000,3.427743,2.000000,...,3.419169,1.000000,84.000000,79.000000,3.346522,2.000000,56.000000,113.000000,3.193213,2.000000
50%,330.000000,3.549556,106.000000,135.000000,3.872031,2.000000,116.000000,102.000000,3.877105,2.000000,...,4.012908,2.000000,108.000000,87.000000,3.817861,2.000000,92.000000,117.000000,3.793730,2.000000
75%,507.000000,5.479259,148.000000,142.000000,4.746920,2.000000,159.000000,108.000000,4.751803,2.000000,...,4.939882,2.000000,150.000000,93.000000,4.652184,2.000000,131.000000,122.000000,4.626660,2.000000
max,1155.000000,9.998607,189.000000,160.000000,8.252149,2.000000,206.000000,121.000000,8.401125,2.000000,...,8.086719,2.000000,199.000000,108.000000,8.263668,2.000000,203.000000,146.000000,8.149048,2.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22100 entries, 0 to 22099
Columns: 222 entries, ID to right_ring2_v
dtypes: float64(222)
memory usage: 37.6 MB


In [ ]:
len(df)

22100

In [ ]:
df.to_csv("fly.csv")